# Etude Statistique

Problématique: Prédiction des entrées dans les cinémas français

Auteur:
- Boukharfien Zainab
- Houssoy Adrien
- Hurdebourcq Paul

Contexte: Nous cherchons à prédire le nombre d'entrées d'un cinéma sur l'année 2020 à l'aide de différentes variables caractéristiques.


## Installation et découverte de l'environement de travail

### Importations des bibliothèques

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importation des données

In [8]:
data=pd.read_csv("../cinematographiques-propre.csv",sep=';',encoding='ANSI')
data.head()

,N° auto,nom,région administrative,adresse,code INSEE,commune,population de la commune,DEP,N°UU,unité urbaine,...,nombre de films en semaine 1,PdM en entrées des films français,PdM en entrées des films américains,PdM en entrées des films européens,PdM en entrées des autres films,films Art et Essai,PdM en entrées des films Art et Essai,latitude,longitude,geolocalisation
0,55,ELYSEES LINCOLN,ILE-DE-FRANCE,14 RUE LINCOLN,75108,Paris 8e Arrondissement,36694,75,851,Paris,...,53,"48,82847702","5,617302493","26,36427356","19,18994693",61,"87,62891759","48,870512","2,303157","48,870512,2,303157"
1,204,UGC OPERA,ILE-DE-FRANCE,32 BD DES ITALIENS,75109,Paris 9e Arrondissement,59408,75,851,Paris,...,41,"47,16468714","42,30156431","10,49210603","0,041642526",33,"53,34226535","48,871332","2,335302","48,871332,2,335302"
2,301,MAX LINDER,ILE-DE-FRANCE,24 BD POISSONNIERE,75109,Paris 9e Arrondissement,59408,75,851,Paris,...,8,"6,97058738","49,98399907","35,58607046","7,459343089",64,"53,67294097","48,871363","2,344886","48,871363,2,344886"
3,451,UGC MONTPARNASSE,ILE-DE-FRANCE,83 BD DU MONTPARNASSE,75106,Paris 6e Arrondissement,42428,75,851,Paris,...,63,"43,58109209","38,32192339","15,69030155","2,406682967",45,"39,64221679","48,84351","2,325646","48,84351,2,325646"
4,731,MK2 ODEON COTE SAINT-GERMAIN,ILE-DE-FRANCE,113 BD ST GERMAIN,75106,Paris 6e Arrondissement,42428,75,851,Paris,...,58,"55,90664627","24,3213105","13,33991217","6,43213105",48,"68,84195984","48,852437","2,338266","48,852437,2,338266"


### Présentation du jeu de données

In [9]:
print(f"Nombre de cinéma étudiées: {data.shape[0]}")
print(f"Nombre de caractéristiques (variable): {data.shape[1]}")

Nombre de cinéma étudiées: 2042
Nombre de caractéristiques (variable): 40


Le jeu de données dont nous disposons regroupe l'activité des cinémas français en 2020. Nous disposons d'une liste de 2042 cinéma et de 40 variables caractéristiques sur les performances du cinéma.
Les variables sont les suivantes:

- N° auto = Identifiant du cinéma
- nom = Nom du cinéma
- région administrative = Nom de la région
- adresse = Adresse du cinéma
- code INSEE = code insee du cinéma
- commune = Commune du cinéma
- population de la commune = Population de la commune du cinéma
- DEP = Numéro de département
- N°UU = Numéro de l'unité urbaine du cinéma
- unité urbaine = Unité urbaine du cinéma
- population unité urbaine = Population de l'unité urbaine du cinéma
- situation géographique = Localisation du cinéma
- écrans = Nombre d'écran dans le cinéma
- fauteuils = Nombre de fauteuils dans le cinéma
- semaines d'activité = Nombres de semaines où le cinéma a été ouvert
- séances = Nombre de séance du cinéma
- entrées 2020 = Nombres d'entrées en 2020
- entrées 2019 = Nombre d'entrée en 2019
- évolution entrées = Evolution des entrées entre 2019 et 2020
- tranche d'entrées = tranche du nombre d'entrée en 2020
- propriétaire = Nom du propriétaire du cinéma
- programmateur = Nom du programmateur des films diffusées
- AE = appartenance à la catégorie art et essai
- catégorie Art et Essai = catégorie des films art et essais
- label Art et Essai = label des film art et essai
- genre = mobilité du cinéma (fixe ou itinérant)
- multiplexe = appartenance à un multiplexe (oui/non)
- zone de la commune = zone de la comune du cinéma
- nombre de films programmés = Nombre de film programmé par le cinéma
- nombre de films inédits = nombre de film inédit diffusé
- nombre de films en semaine 1 = nombre de film en semaine 1 diffusé
- PdM en entrées des films français = part de marché des films français dans le cinéma
- PdM en entrées des films américains = part de marché des films américains dans le cinéma
- PdM en entrées des films européens = part de marché des films européens dans le cinéma
- PdM en entrées des autres films = part de marché des autres films dans le cinéma
- films Art et Essai = nombre de film art et essaie diffusé
- PdM en entrées des films Art et Essai = part de marché des films art et essai dans le cinéma
- latitude = latitude geographique du cinéma
- longitude = longitude geographqiue du cinéma
- geolocalisation = coordonnées geographique du cinéma



 ## Analyse exploiratoire

### Pré-traitement

In [15]:
def convertion(data, typ, *var):
    """
    Fonction qui modifie le type des variables
    
    : param list data : les donnees
    : return : les data avec le bon type pour chaque var
    : rtype : list
    """
    types = ["str","int","float"]
    while typ not in types:
        typ = input("Type non reconnu, merci de choisir entre \"str\", \"int\" ou \"float\" \n►")
    for ele in var:
        if typ == "str":
            for i in range(1, len(data)):
                data[i][index] = str(data[i][index])
        if typ == "int":
            for i in range(1, len(data)):
                data[i][index] = int(data[i][index])
        if typ == "float":
            for i in range(1, len(data)):
                data[i][index] = float(data[i][index])
    return data

In [16]:
data = convertion(data, "str", 'Numéro_Auto','Nom','Adresse','Région Admnistratives','Commune','N°INSEE', 'Departement', 'Nom Unité urbaine', 'propriétaire_cinéma','Art_et_essai', 'cat_art-essaie','label_art-essaie','mobilité','multiplexe','coordonnée_geo',"programmateur")
data = convertion(data, "int", 'Population','Population Unité urbaine','Ecran', 'Nb_Fauteuils', 'Nb_semaines_ouvertes','Nb_Séance','Nb_Entrées-2020','Nb_Entrées-2019', 'nb_programmation','nb_films_inédit','nb_sem1')
data = convertion(data, "float", 'Evolution_entrees', 'pdm_film_fr', 'pdm_film_US', 'pdm_film_UE', 'pdm_film_autres', 'pdm_fil_art-essai')

KeyError: 1

In [10]:
def tri_var(df):
  """
  Fonction qui sépare les variables qualitatives et quantitatives
  Param: 
    - data (Dataframe): jeu de données initial
  Return:
    - quali (liste) : liste des variables qualitatives
    - quanti (liste) :  liste des variables quantitatives
  """
  quanti=[]
  quali=[]
  df_col=df.columns
  for i in df_col:
    if 'float' in str(df[i].dtypes):
      quanti.append(i)
    else:
      quali.append(i)

  return quali,quanti

In [11]:
var_quali, var_quanti = tri_var(data)

N° auto
nom
région administrative
adresse
code INSEE
commune
population de la commune
DEP
N°UU
unité urbaine
population unité urbaine
situation géographique
écrans
fauteuils
semaines d'activité
séances
entrées 2020
entrées 2019
évolution entrées
tranche d'entrées
propriétaire
programmateur
AE
catégorie Art et Essai
label Art et Essai
genre
multiplexe
zone de la commune
nombre de films programmés
nombre de films inédits
nombre de films en semaine 1
PdM en entrées des films français
PdM en entrées des films américains
PdM en entrées des films européens
PdM en entrées des autres films
films Art et Essai
PdM en entrées des films Art et Essai
latitude
longitude
geolocalisation
